In [0]:
-- 1. Create source table in scd_cat.scd_schema: This table holds the latest incoming data snapshot.
CREATE OR REPLACE TABLE scd_cat.scd_schema.source_table (
  id INT,
  name STRING,
  value STRING,
  updated_at TIMESTAMP
);

insert into scd_cat.scd_schema.source_table values (1, 'a', '1', '2023-01-01 00:00:00'),(2,'b','2','2023-01-02 00:00:00');

In [0]:
select * from scd_cat.scd_schema.source_table

In [0]:
-- 2. Create destination (SCD2) table in scd_cat.scd_schema: This table tracks historical changes using SCD Type 2 logic.
CREATE OR REPLACE TABLE scd_cat.scd_schema.scd2_table (
  id INT,
  name STRING,
  value STRING,
  valid_from TIMESTAMP,      -- When this record became valid
  valid_to TIMESTAMP,        -- When this record was superseded (or max date if current)
  is_current BOOLEAN         -- TRUE if this is the current record for the id
);

In [0]:
-- 3. SCD2 Merge Logic: Update or insert records based on changes in source data.
MERGE INTO
  scd_cat.scd_schema.scd2_table AS target
USING
  scd_cat.scd_schema.source_table AS source
ON
  target.id = source.id
  AND target.is_current = TRUE
WHEN MATCHED AND
  (
    target.name <> source.name
    OR target.value <> source.value
  )
  THEN UPDATE SET target.valid_to = source.updated_at, target.is_current = FALSE
WHEN NOT MATCHED BY TARGET THEN INSERT (id, name, value, valid_from, valid_to, is_current)
  VALUES (
    source.id,
    source.name,
    source.value,
    source.updated_at,
    TIMESTAMP '9999-12-31 23:59:59',
    TRUE
  );

-- 4. Insert new current records for updated/expired ones:
INSERT INTO scd_cat.scd_schema.scd2_table
  SELECT
    s.id,
    s.name,
    s.value,
    s.updated_at AS valid_from,
    TIMESTAMP '9999-12-31 23:59:59' AS valid_to,
    TRUE AS is_current
  FROM
    scd_cat.scd_schema.source_table s
      LEFT JOIN scd_cat.scd_schema.scd2_table t
        ON s.id = t.id
        AND t.is_current = FALSE
        AND t.valid_to = s.updated_at
  WHERE
    t.id IS NOT NULL;

In [0]:
select * from scd_cat.scd_schema.scd2_table

In [0]:
update
  scd_cat.scd_schema.source_table
set
  name = 'ab'
where
  id = 1